In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
file = '../input/spam-data/train.csv'
dataset = pd.read_csv(file)

In [ ]:
dataset

In [ ]:
# def to_lower(word):
#     result = word.lower()
#     return result

# def remove_special_characters(word):
#     result= word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
#     return result

# def remove_stop_words(words):
#     result = [i for i in words if i not in ENGLISH_STOP_WORDS]
#     return result

# def remove_hyperlink(word):
#     return re.sub(r"http\S+", "", word)


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

texts = []
for i in range(0,1500):
    mail = re.sub('[^a-zA-Z]',' ',dataset['question_text'][i])
    mail = mail.lower()
    mail = mail.split()
    ps = PorterStemmer()
    mail = [ps.stem(word) for word in mail 
            if not word in set(stopwords.words('english','french'))]
    mail = ' '.join(mail)
    texts.append(mail)
    print(texts)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(texts).toarray()
y = dataset.iloc[:1500,-1].values

In [ ]:
y[1:1000]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(x_train,maxlen=40)
x_test = pad_sequences(x_test,maxlen=40)

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_test = le.transform(y_test)

In [ ]:
import tensorflow as tf
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

#size of the output vector from each layer
embedding_vector_length = 32

 

In [ ]:
max_feature = 1500
#Creating a sequential model
model = tf.keras.Sequential()

#Creating an embedding layer to vectorize
model.add(Embedding(max_feature, embedding_vector_length, input_length=40))

#Addding Bi-directional LSTM
model.add(Bidirectional(tf.keras.layers.LSTM(64)))

In [ ]:
#Relu allows converging quickly and allows backpropagation
model.add(Dense(16, activation='relu'))

#Deep Learninng models can be overfit easily, to avoid this, we add randomization using drop out
model.add(Dropout(0.1))

#Adding sigmoid activation function to normalize the output
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

 

In [ ]:
history = model.fit(x_train, y_train, batch_size=512, epochs=20, validation_data=(x_test, y_test))

y_predict = [1 if o>0.5 else 0 for o in model.predict(x_test)]


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

cf_matrix =confusion_matrix(y_test,y_predict)

print(cf_matrix)

score =accuracy_score(y_test,y_predict)

print(score)
